# WINA

Revisión informal de _WINA_ (_Weight Informed Neuron Activation for Accelerating Large Language Model Inference_).

Técnica publicada por Microsoft en mayo de 2025.

- [https://arxiv.org/pdf/2505.19427](https://arxiv.org/pdf/2505.19427)

Desactiva neuronas de manera selectiva (_sparse activation_), sin necesidad de reentrenar los modelos.

## 1. Arquitectura

Las técnicas habituales utilizan únicamente las magnitudes de las entradas para decidir qué neuronas desactivar.

```
Input ─> Gate ─> Layer ─> Output
```

_WINA_ utiliza las magnitudes de las entradas y de los pesos.

```
Input ─> Gate ─> Layer ─> Output
          ↑        ↓
          └──── Weights
```

## 2. Análisis

Propone una expresión muy sencilla para la función de puerta (_gate_) de las capas, y demuestra matemáticamente su superioridad frente a otros métodos, pero impone condiciones a la hora de evaluar los modelos que deben conocerse.

### 2.1. Función Objetivo

El marco teórico del _paper_ parte de la definición de los elementos que componen una capa y la operación básica que realiza.

Siendo:

- $W \in \mathbb{R}^{m \times n}$ la matriz de pesos de la capa.

- $x \in \mathbb{R}^{n}$ un vector arbitrario de entrada a la capa.

- $y \leftarrow W x$ la transformación lineal estándar que multiplica pesos por entrada para obtener la salida.

Plantea el problema de desactivar neuronas como aplicar una máscara binaria sobre la entrada.

Siendo:

- $m \in \{0, 1\}^n$ el vector que enmascara la entrada.

- $y_m \leftarrow W (m \odot x)$ la salida de la capa aplicando la máscara $m$ a la entrada.

Y define el objetivo como encontrar la máscara $m$ que minimice la diferencia entre la salida original y la nueva aplicando dicha máscara.

- $ \underset{m \in \{0, 1\}^n}{\text{arg min}} \lVert W x - W (m \odot x) \rVert^2_2 $

### 2.2. Binary Activation Gate

Los elementos $m_i$ de la máscara se calculan utilizando la entrada y los pesos.

-  $ m_i = \begin{cases} 1 \quad \text{si} \mid x_i c_i \mid \text{está entre los top-}K \text{valores en} \mid x \odot \ c \mid \\ 0 \quad \text{en cualquier otro caso} \end{cases} $

Siendo:

- $c$ el vector resultante de calcular la norma L2 columna a columna sobre la matriz de pesos.

- $c_i = \lVert W_{\cdot,i} \rVert_2$ el elemento $i$-ésimo resultante de calcular la norma L2 sobre la columna $i$ de la matriz de pesos.

- $\mid x \odot c \mid$ el producto de la entrada por las normas, elemento a elemento, en valor absoluto.

- $\mid x_i c_i \mid$ el elemento $i$-ésimo del vector de los productos de la entrada por las normas.

- $\text{top-}K$ el subconjunto de los $K$ valores mayores del vector $\mid x \odot c \mid$.

- $K$ un valor escalar fijo que define el número de entradas que se deben dejar pasar.

$K$ puede tener el mismo valor para todas las capas, o definirse de forma más específica para cada capa.

### 2.3. Single Layer

Para demostrar las bondades de _WINA_, se compara con _TEAL_, otra técnica similar de referencia, que sólo tiene en cuenta los valores absolutos del vector de entrada.

El _paper_ establece que si la matriz de pesos tiene columnas ortonormales.

- $ W^T W = I_n $

Entonces se verifica que el error cometido por _WINA_ es menor o igual que el cometido por _TEAL_ para cualquier nivel $K < n$ de activación deseado.

- $ \mathbb{E} \left[ \lVert W x_{\text{WINA}} - W x \rVert^2_2 \right] \le \mathbb{E} \left[ \lVert W x_{\text{TEAL}} - W x \rVert^2_2 \right] $

Donde $W x_{\text{WINA}}$ es la salida de la capa con _WINA_, y $W x_{\text{TEAL}}$ la salida con _TEAL_.

En los apéndices del _paper_ puede encontrarse la demostración formal.

### 2.4. Multiple Layers

Suponiendo que todas las matrices de pesos de una red, con cualquier número de capas, tienen columnas ortonormales.

Se verifica que el error cometido por _WINA_ es menor o igual que el cometido por _TEAL_ a la salida de la red para cualquier nivel $K_i < n$ de activación deseado.

- $ \mathbb{E} \left[ \lVert y_{\text{WINA}} - y \rVert^2_2 \right] \le \mathbb{E} \left[ \lVert y_{\text{TEAL}} - y \rVert^2_2 \right] $

Donde $y_{\text{WINA}}$ es el resultado obtenido por el modelo con _WINA_, e $y_{\text{TEAL}}$ el obtenido con _TEAL_.

En los apéndices del _paper_ puede encontrarse la demostración formal.

### 2.5. Activation Layer

Establece que si la matriz de pesos de una capa con una función de activación tiene columnas ortonormales.

Y la función de activación es monotónica no decreciente, es decir, que su salida no disminuye cuando aumenta su entrada.

- $f: \mathbb{R} \rightarrow \mathbb{R}, \quad \forall x_1 \le x_2 \quad f(x_1) \le f(x_2) $

Entonces se verifica que el error cometido por _WINA_ es menor o igual que el cometido por _TEAL_ para cualquier nivel $K < n$ de activación deseado.

- $ \mathbb{E} \left[ \lVert f(W x_{\text{WINA}}) - f(W x) \rVert^2_2 \right] \le \mathbb{E} \left[ \lVert f(W x_{\text{TEAL}}) - f(W x) \rVert^2_2 \right] $

Ejemplos de este tipo de funciones de activación son _ReLU_, _LeakyReLU_, y cualquiera de sus variantes.

En los apéndices del _paper_ puede encontrarse la demostración formal.

### 2.6. Multiple Activation Layers

Suponiendo que todas las matrices de pesos de una red, con cualquier número de capas, tienen columnas ortonormales.

Y todas las funciones de activación son monotónicas no decrecientes.

Se verifica que el error cometido por _WINA_ es menor o igual que el cometido por _TEAL_ a la salida de la red para cualquier nivel $K_i < n$ de activación deseado.

- $ \mathbb{E} \left[ \lVert y_{\text{WINA}} - y \rVert^2_2 \right] \le \mathbb{E} \left[ \lVert y_{\text{TEAL}} - y \rVert^2_2 \right] $

En los apéndices del _paper_ puede encontrarse la demostración formal.

### 2.7. Transformation Protocol

La mayor limitación impuesta por la técnica es la condición de que todas las matrices de pesos deben tener columnas ortonormales, lo que no se puede garantizar en modo alguno.

- $ W^T W = I_n $

El _paper_ propone en este punto aplicar una transformación a las matrices de pesos.

Utilizando _Singular Value Decomposition_ (_SVD_) descompone la matriz $W$ en tres matrices.

- $ W = U \Sigma V^T $

Donde $U$ y $V$ son matrices con columnas ortonormales, y $\Sigma$ es una matriz diagonal que contiene los valores singulares de $W$.

Y realiza el producto de $W$ por $V$ que produce una nueva matriz $W'$ con columnas ortogonales, pero no necesariamente ortonormales.

- $ W' = W V $

### 2.8. Computational Invariance

Los cambios en las matrices de pesos cambian la salida del modelo, por lo que es necesario aplicar medidas para compensarlos.

El _paper_ propone aplicar la técnica conocida como _computational invariance_, que establece que se puede aplicar una transformación ortogonal a la salida de un componente siempre que se deshaga en el siguiente.

Por lo que, finalmente, para aplicar _WINA_ se debe forzar la condición impuesta a las matrices más relevantes de la red, como son las clásicas matrices de pesos $W_K$ del mecanismo de atención y $W_{gate}$ de _MLP_. Aplicar la técnica de _computacional invariance_ para propagar y ajustar el efecto  de los cambios realizados. Y utilizar la técnica basada en la norma L2 en estas matrices, y la basada únicamente en las magnitudes de las entradas en el resto.